Combining All History JSON Files

In [ ]:
import pandas as pd

#combining total history
history0 = pd.read_json('syd0.json')
history1 = pd.read_json('syd1.json')
history2 = pd.read_json('syd2.json')
history3 = pd.read_json('syd3.json')
history4 = pd.read_json('syd4.json')
history5 = pd.read_json('syd5.json')
history6 = pd.read_json('syd6.json')
history7 = pd.read_json('syd7.json')
history8 = pd.read_json('syd8.json')
history9 = pd.read_json('syd9.json')
history10 = pd.read_json('syd10.json')
history11 = pd.read_json('syd11.json')
history12 = pd.read_json('syd12.json')
history13 = pd.read_json('syd13.json')
history14 = pd.read_json('syd14.json')
history15 = pd.read_json('syd15.json')
history16 = pd.read_json('syd16.json')
history17 = pd.read_json('syd17.json')
history18 = pd.read_json('syd18.json')
history19 = pd.read_json('syd19.json')
complete = pd.concat([history0,history1,history2,history3,history4,history5,history6,history7,history8,history9,history10,history11,history12,history13,history14,history15,history16,history17,history18,history19])
print(complete.columns)







Cleaning and Reformatting/naming Columns

In [ ]:
#Drop unecessary columns and rename original
complete = complete.drop(columns=['platform', 'ip_addr', 'audiobook_title', 'audiobook_uri', 'audiobook_chapter_uri', 'audiobook_chapter_title', 'offline', 'offline_timestamp'])
complete = complete.rename(columns={'ts':'exactTime', 'ms_played':'ms', 'conn_country':'country','master_metadata_track_name':'trackName',	'master_metadata_album_artist_name':'artistName','master_metadata_album_album_name':'albumName'	,'spotify_track_uri':'trackURI', 'episode_name':'episodeName','episode_show_name':'showName', 'spotify_episode_uri':'episodeURI','reason_start':'startReason','reason_end':'endReason','incognito_mode':'incognito'})

#convert ms to minutes
complete['duration'] = complete['ms']/60000
#extract specific times
complete['exactTime'] = pd.to_datetime(complete['exactTime'])
complete['hour'] = complete['exactTime'].dt.hour
complete['day'] = complete['exactTime'].dt.date
complete['month'] = complete['exactTime'].dt.month
complete['weekday'] = complete['exactTime'].dt.weekday
complete['year'] = complete['exactTime'].dt.year

complete.head() # complete['year'] = complete['year'].dt.year


Split Tracks and Episodes

In [ ]:
#Split up dataframe into tracks and episodes/podcasts

episodes = complete[complete['episodeName'].notnull()] #save rows where there's an episodeName
tracks = complete[complete['trackName'].notnull()] #save rows where there's a trackName
print(len(episodes))
print(len(tracks))
print(len(complete))
#tracks = complete[~complete['trackName'].notnull()]



Store all rows with unique artists

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
from dotenv import load_dotenv
import os
client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))
#only keep unique artists
uniqueArtists = tracks.drop_duplicates(subset=['artistName'])
uniqueArtists.head()

print(len(uniqueArtists))



Add track history to postgres


In [ ]:

from sqlalchemy import create_engine
from sqlalchemy import inspect

 #connection to postgres db
conn_string = 'postgresql://postgres:' + os.getenv('postgres_pass')  + '@localhost:' + os.getenv('postgres_port') + '/Spotify'
engine = create_engine(conn_string)
#validate if it works
inspector = inspect(engine)
print(inspector.get_table_names(schema='public'))



In [ ]:



# trackInfo = sp.track("spotify:track:50a8bKqlwDEqeiEknrzkTO") #Get spotify trackinfo
# artist = trackInfo['artists'][0] 
# artistID = artist['id'] #Get artist ID from artist objet
# genres =  sp.artist(artistID)['genres'] #Get genre(s) of artists
# print(genres)
#Dict to map artist with genres
# artistGenre = {}
# for t in uniqueArtists['trackURI']:
#     try: 
#         trackInfo = sp.track(t) #Get spotify trackinfo
#         artist = trackInfo['artists'][0] 
#         artistID = artist['id'] #Get artist ID from artist objet
#         genres =  sp.artist(artistID)['genres'] #Get genre(s) of artists
#         if artist['name'] not in artistGenre:
#             artistGenre[artist['name']] = genres
#     except Exception as e: #If artist is invalid print error  ``
#         print(e)
#         continue
# print(len(artistGenre))
    
# #print(top)
# # print(client_id)
# # print(client_secret)

